In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.io import loadmat
%matplotlib inline

In [242]:
from IPython.core.debugger import Tracer

In [8]:
import scipy.misc

In [3]:
infile=loadmat('ex4data1.mat')

In [5]:
infile.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [6]:
X=infile['X'].copy()
y=infile['y'].copy()

In [7]:
X.shape,y.shape

((5000, 400), (5000, 1))

In [14]:
a=X[1000,:].reshape((20,20))

In [15]:
scipy.misc.toimage(a)

In [16]:
infile2=scipy.io.loadmat('ex4weights.mat')

In [18]:
infile2.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Theta1', 'Theta2'])

In [19]:
theta1=infile2['Theta1'].copy()

In [20]:
theta2=infile2['Theta2'].copy()

In [21]:
theta1.shape

(25, 401)

In [22]:
theta2.shape

(10, 26)

In [26]:
theta=np.concatenate((theta1.flatten(),theta2.flatten()))

In [27]:
theta.shape

(10285,)

In [37]:
hiddenlayer=25
inputlayer=400
outputlayer=10
theta1_len=hiddenlayer*(inputlayer+1)
tmp1=theta[:theta1_len].reshape((hiddenlayer,(inputlayer+1)))
tmp2=theta[theta1_len:].reshape((outputlayer,(hiddenlayer+1)))

In [195]:
def random_ini(inputlayer,outputlayer,epsilon):
    W=np.random.rand(outputlayer,inputlayer+1)*2*epsilon-epsilon
    return W

In [163]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [182]:
def sigmoid_grad(x):
    return sigmoid(x)*(1-sigmoid(x))

In [271]:
def nncost(theta,inputlayer,hiddenlayer,outputlayer,X,y,lamb):
    theta1_len=hiddenlayer*(inputlayer+1)
    theta1=theta[:theta1_len].reshape((hiddenlayer,(inputlayer+1)))
    theta2=theta[theta1_len:].reshape((outputlayer,(hiddenlayer+1)))
    total_cost=0
    m=X.shape[0]
    X=np.insert(X,0,1,axis=1)
    a2=sigmoid(theta1.dot(X.T))
    a2=np.insert(a2,0,1,axis=0)
    a3=sigmoid(theta2.dot(a2))
    total_cost=0
    for i in range(m):
        tmp=np.zeros((outputlayer,1))
        j=y[i,0]-1
        tmp[j,0]=1
        cost=-(tmp.T.dot(np.log(a3[:,i])))-((1-tmp).T.dot(np.log(1-a3[:,i])))
        total_cost += cost
    theta1_cost=theta1[:,1:]
    theta2_cost=theta2[:,1:]
    J=total_cost/m+lamb/(2*m)*((theta1_cost**2).sum()+(theta2_cost**2).sum())
    return J[0]

In [294]:
def nngrad(theta,inputlayer,hiddenlayer,outputlayer,X,y,lamb):
    theta1_len=hiddenlayer*(inputlayer+1)
    theta1=theta[:theta1_len].reshape((hiddenlayer,(inputlayer+1)))
    theta2=theta[theta1_len:].reshape((outputlayer,(hiddenlayer+1)))
    theta1_grad=np.zeros_like(theta1)
    theta2_grad=np.zeros_like(theta2)
#     Tracer()()
    X=np.insert(X,0,1,axis=1)
    m=X.shape[0]
    X_trans=X.T
    for i in range(m):
        a1=X_trans[:,[i]]
        z2=theta1.dot(a1)
        a2=sigmoid(z2)
        a2=np.insert(a2,0,1,axis=0)
        z3=theta2.dot(a2)
        a3=sigmoid(z3)
        #in the following three lines y (an integer between 1 to 10 is translated into a
        #vector of 10. The y th member (indexed y-1) of this vector is set to be one, rest zero.)
        tmp=np.zeros((outputlayer,1))
        j=y[i,0]-1
        tmp[j,0]=1
        delta_3=a3-tmp
        delta_2=theta2.T.dot(delta_3)*(a2*(1-a2))
        delta_2=delta_2[1:,:]
        theta1_grad += delta_2.dot(a1.T)
        theta2_grad += delta_3.dot(a2.T)
    theta1_reg=theta1.copy()
    theta1_reg[:,0]=0
    theta2_reg=theta2.copy()
    theta2_reg[:,0]=0
    theta1_grad=theta1_grad/m+(lamb/m)*theta1_reg
    theta2_grad=theta2_grad/m+(lamb/m)*theta2_reg
    return np.concatenate((theta1_grad.flatten(),theta2_grad.flatten()))

In [296]:
nngrad(theta,400,25,10,X,y,1)

array([  6.18712766e-05,  -2.11248326e-12,   4.38829369e-13, ...,
         4.70513145e-05,  -5.01718610e-04,   5.07825789e-04])

Do a test case:

In [286]:
def gradient_check(theta,epsilon,X,y,lamb):
    grad=np.zeros_like(theta)
    for i in range(len(theta)):
        pert=np.zeros_like(theta)
        pert[i]=epsilon
        p1=theta+pert
        p2=theta-pert
        loss1=nncost(p1,400,25,10,X,y,lamb)
        loss2=nncost(p2,400,25,10,X,y,lamb)
        grad[i]=(loss1-loss2)/(2*epsilon)
    return grad
    

In [262]:
X_test=X[[0],:]

In [263]:
X_test.shape

(1, 400)

In [289]:
grad_test_1=nngrad(theta,400,25,10,X_test,y,0)

In [290]:
grad_test_2=gradient_check(theta,1e-4,X_test,y,0)

In [291]:
(grad_test_1-grad_test_2).max()

1.2299149546035615e-11

In [297]:
grad_test_3=nngrad(theta,400,25,10,X_test,y,10)
grad_test_4=gradient_check(theta,1e-4,X_test,y,10)

In [298]:
(grad_test_3-grad_test_4).max()

9.7092751616401074e-09

In [299]:
grad_test_3.max()

32.123786623742639

In [176]:
nncost(theta,400,25,10,X,y,0)

0.28762916516131881

In [178]:
nncost(theta,400,25,10,X,y,1)

0.38376985909092354

In [301]:
res=minimize(nncost,theta,method="BFGS",args=(400,25,10,X,y,1),jac=nngrad)

KeyboardInterrupt: 